In [47]:
import numpy as np
import random
import math


In [48]:
# la bound
# lb bound
# ht bound


In [66]:
def gen_deadline(exec_time, period):
    # [a,b]
    b = 1.2 * period
    if(exec_time < 10):
        a = exec_time
    elif(exec_time < 100 and exec_time >= 10):
        a = 2*exec_time
    elif(exec_time < 1000 and exec_time >= 100):
        a = 3*exec_time
    else:
        a = 4*exec_time

    return a+random.random()*(b-a)


def gen_tasksets(utilizations, periods):
    """
    Take a list of task utilization sets and a list of task period sets and
    return a list of couples (c, p) sets. The computation times are truncated
    at a precision of 10^-10 to avoid floating point precision errors.
    Args:
        - `utilization`: The list of task utilization sets. For example::
            [[0.3, 0.4, 0.8], [0.1, 0.9, 0.5]]
        - `periods`: The list of task period sets. For examples::
            [[100, 50, 1000], [200, 500, 10]]
    Returns:
        For the above example, it returns::
            [[(30.0, 100), (20.0, 50), (800.0, 1000)],
             [(20.0, 200), (450.0, 500), (5.0, 10)]]
    """
    def trunc(x, p):
        return int(x * 10 ** p) / float(10 ** p)
    [[print(ui) for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)]
    return np.array([[[trunc(ui * pi, 6), trunc(pi, 6), trunc(gen_deadline(trunc(ui * pi, 6), trunc(pi, 6)), 6)] for ui, pi in zip(us, ps)]
            for us, ps in zip(utilizations, periods)])


In [67]:
def custom_period(n, nsets,max_,min_=0):
	max_power = np.log(max_)


	intervals = np.arange(0,np.floor(max_power)+1)
	intervals = np.append(intervals, max_power)
	#print(intervals)
	task_pp = ((n-1)//(len(intervals)-1))
	
	task_lf = np.mod(n-1, len(intervals)-1).astype(int)
	#print(task_lf,task_pp)
	periods = np.zeros((nsets, n))
	for i in range(nsets):
		for j in range(len(intervals)-1):
			if(j<task_lf):
				periods[i][j*task_pp+j:(j+1)*task_pp+j+1] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp+1)
			else:
				periods[i][j*task_pp+task_lf:(j+1)*task_pp+task_lf] = np.random.uniform(low=np.exp(intervals[j]), high=np.exp(intervals[j+1]), size=task_pp)
	periods[0:nsets,-1] =max_ 
	return periods


In [68]:
custom_period(14,2,100)

array([[  2.45175765,   1.43094888,   1.79767535,   3.98048456,
          2.97195392,   6.8300681 ,   8.47884653,   8.57980871,
         17.06926471,  49.59793226,  45.94429206,  89.54683752,
         77.64738142, 100.        ],
       [  1.05955495,   1.5444694 ,   1.74983236,   6.202584  ,
          5.66637107,   2.74497973,  19.72203596,  12.73266528,
         18.96583292,  39.85565407,  35.24353688,  65.09466348,
         67.32289707, 100.        ]])

In [69]:
def uunifast(n, u):
    """
    Generate a random set of `n` utilizations that sum up to `u` using the
    uunifast algorithm.
    Args:
	- `n`: The number of utilizations to generate.
	- `u`: The sum of the utilizations.
    """
    us = [0.0] * n
    us[0] = u
    for i in range(1, n):
        us[i] = np.random.uniform(low=0.0, high=us[i - 1])
    us.sort()
    us = [us[i] - us[i - 1] for i in range(1, n)]
    us.append(us[0] + us[-1])
    return us

In [70]:
def UUniFastDiscard(n, u, nsets):
    sets = []
    while len(sets) < nsets:
        # Classic UUniFast algorithm:
        utilizations = []
        sumU = u
        for i in range(1, n):
            nextSumU = sumU * random.random() ** (1.0 / (n - i))
            utilizations.append(sumU - nextSumU)
            sumU = nextSumU
        utilizations.append(sumU)

        # If no task utilization exceeds 1:
        if all(ut <= 1 for ut in utilizations):
            sets.append(utilizations)

    return sets

    
def gen_uunifastdiscard(nsets, u, n):
    """
    The UUniFast algorithm was proposed by Bini for generating task
    utilizations on uniprocessor architectures.
    The UUniFast-Discard algorithm extends it to multiprocessor by
    discarding task sets containing any utilization that exceeds 1.
    This algorithm is easy and widely used. However, it suffers from very
    long computation times when n is close to u. Stafford's algorithm is
    faster.
    Args:
        - `n`: The number of tasks in a task set.
        - `u`: Total utilization of the task set.
        - `nsets`: Number of sets to generate.
    Returns `nsets` of `n` task utilizations.
    """
    return UUniFastDiscard(n, u, nsets)

In [71]:
def get_taskset(nsets):
	u = 0.9
	n = 5
	utilizations = gen_uunifastdiscard(nsets, u, n)
	periods = custom_period(n, nsets, 1000, 10)
	return gen_tasksets(utilizations, periods)

	

In [83]:
k = get_taskset(2)
z = k[0]
print(k)
print(z[:,2])
#gen_uunifastdiscard(u=0.9, n=5, nsets=1)

0.07414867637935163
0.4452596906299144
0.08228340177306426
0.12614214832541304
0.1721660828922567
0.3473800846141528
0.16271132931408439
0.04068354039967348
0.13631717855111503
0.21290786712097431
[[[9.09400000e-02 1.22646800e+00 4.56303000e-01]
  [1.87317000e+00 4.20691600e+00 3.29020100e+00]
  [1.11526900e+00 1.35540080e+01 3.68618300e+00]
  [6.45508600e+00 5.11731110e+01 1.28011010e+01]
  [1.72166082e+02 1.00000000e+03 9.57485882e+02]]

 [[6.39523000e-01 1.84099000e+00 2.05908000e+00]
  [7.71622000e-01 4.74228000e+00 2.00129800e+00]
  [5.16927000e-01 1.27060600e+01 1.12854900e+01]
  [4.66770900e+00 3.42415360e+01 1.50371030e+01]
  [2.12907867e+02 1.00000000e+03 8.89856681e+02]]]
[4.56303000e-01 3.29020100e+00 3.68618300e+00 1.28011010e+01
 9.57485882e+02]


In [73]:
def qpa_algorithm(taskset):
	"""
	Compute h(t) calculations for the QPA algorithm
	
	"""
	

In [91]:
def La_bound(taskset):
	"""
	Compute La calculations for the La bound
	
	"""
	
	deadlines = taskset[:,2]
	periods = taskset[:,1]
	utilizations = taskset[:,0]/taskset[:,1]
	#print(deadlines, periods, utilizations)
	total_utilization = np.sum(utilizations)
	#print(np.sum((periods-deadlines)*utilizations)/(1-total_utilization))
	La = np.maximum(deadlines,np.sum((periods-deadlines)*utilizations)/(1-total_utilization))

	


In [92]:
bound = La_bound(k[0])

[4.56303000e-01 3.29020100e+00 3.68618300e+00 1.28011010e+01
 9.57485882e+02] [   1.226468    4.206916   13.554008   51.173111 1000.      ] [0.07414788 0.44525966 0.08228334 0.12614215 0.17216608]
134.36936379520307
